In [1]:
import sys
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from datetime import datetime
from itertools import combinations
PREDICCION_REAL = True
MAXIMIZAR_HIPERPARAMETROS = False
PARAMETROS = {"booster":"gbtree", "max_depth":3, "eta": 0.1, "objective": "binary:logistic", "nthread":2,"gamma" : 0,"subsample" : 1}
RONDAS = 500
EARLY = 1

In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_csv("entrenamiento-listo.csv")
validacion = pd.read_csv("validacion-listo.csv")
test = pd.read_csv("test-listo.csv")

In [3]:
#FILTRADO DE COLUMNAS - NO REMOVER STAGE O FECHA
entrenamiento = entrenamiento.drop(columns=['ID','Opportunity_Name','Sales_Contract_No'])
test = test.drop(columns=['ID','Opportunity_Name','Sales_Contract_No'])
validacion = validacion.drop(columns=['ID','Opportunity_Name','Sales_Contract_No'])

In [4]:
if(PREDICCION_REAL):
    objetivo = (entrenamiento['Stage'] == 'Closed Won').astype(int)
    objetivo_val = (validacion['Stage'] == 'Closed Won').astype(int)
    entrenamiento = entrenamiento.drop(columns=['Stage','Fecha'])
    validacion = validacion.drop(columns=['Stage','Fecha'])
    
    d_entrenamiento = xgb.DMatrix(entrenamiento.values, objetivo.values)
    d_prueba = xgb.DMatrix(test.values)
    d_validacion = xgb.DMatrix(validacion.values, objetivo_val.values)
    
    evaluacion = [(d_validacion, 'eval'), (d_entrenamiento, 'train')]
    
    bst = xgb.train(PARAMETROS, d_entrenamiento, RONDAS,evaluacion,early_stopping_rounds = EARLY)
    preds = bst.predict(d_prueba)

    resultados = test[['Opportunity_ID']].copy()
    resultados['Target'] = pd.Series(preds)
    #resultados = resultados.groupby('Opportunity_ID').mean()
    #resultados = resultados.reset_index()
    #resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))
    
    resultados.to_csv("prediccion.csv", index=False)
    print(resultados['Target'].count())

[14:45:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	eval-logloss:0.67829	train-logloss:0.64753
[1]	eval-logloss:0.66844	train-logloss:0.61013
[2]	eval-logloss:0.66442	train-logloss:0.57852
[3]	eval-logloss:0.64399	train-logloss:0.54959
[4]	eval-logloss:0.63515	train-logloss:0.52738
[5]	eval-logloss:0.63098	train-logloss:0.50598
[6]	eval-logloss:0.62022	train-logloss:0.48836
[7]	eval-logloss:0.62288	train-logloss:0.47180
[8]	eval-logloss:0.62354	train-logloss:0.45806
[9]	eval-logloss:0.63073	train-logloss:0.44524
[10]	eval-logloss:0.62842	train-logloss:0.43317
[11]	eval-logloss:0.61521	train-logloss:0.42357
[12]	eval-logloss:0.60735	train-logloss:0.41375
[13]	eval-logloss:0.60669	train-logloss:0.40556
[14]	eval-logloss:0.59679	tr

[164]	eval-logloss:0.57707	train-logloss:0.23572
[165]	eval-logloss:0.57623	train-logloss:0.23549
[166]	eval-logloss:0.57658	train-logloss:0.23472
[167]	eval-logloss:0.57684	train-logloss:0.23443
[168]	eval-logloss:0.57695	train-logloss:0.23403
[169]	eval-logloss:0.57629	train-logloss:0.23356
[170]	eval-logloss:0.57797	train-logloss:0.23334
[171]	eval-logloss:0.57777	train-logloss:0.23298
[172]	eval-logloss:0.57810	train-logloss:0.23244
[173]	eval-logloss:0.57583	train-logloss:0.23190
[174]	eval-logloss:0.57719	train-logloss:0.23136
[175]	eval-logloss:0.57682	train-logloss:0.23053
[176]	eval-logloss:0.57683	train-logloss:0.22990
[177]	eval-logloss:0.57465	train-logloss:0.22927
[178]	eval-logloss:0.57072	train-logloss:0.22884
[179]	eval-logloss:0.57241	train-logloss:0.22848
[180]	eval-logloss:0.56985	train-logloss:0.22823
[181]	eval-logloss:0.56986	train-logloss:0.22800
[182]	eval-logloss:0.56854	train-logloss:0.22763
[183]	eval-logloss:0.56702	train-logloss:0.22715
[184]	eval-logloss:0

[332]	eval-logloss:0.59436	train-logloss:0.18503
[333]	eval-logloss:0.59413	train-logloss:0.18477
[334]	eval-logloss:0.59639	train-logloss:0.18446
[335]	eval-logloss:0.59644	train-logloss:0.18422
[336]	eval-logloss:0.59681	train-logloss:0.18397
[337]	eval-logloss:0.59635	train-logloss:0.18387
[338]	eval-logloss:0.59691	train-logloss:0.18372
[339]	eval-logloss:0.59704	train-logloss:0.18351
[340]	eval-logloss:0.59496	train-logloss:0.18326
[341]	eval-logloss:0.59321	train-logloss:0.18317
[342]	eval-logloss:0.59323	train-logloss:0.18304
[343]	eval-logloss:0.59306	train-logloss:0.18290
[344]	eval-logloss:0.59279	train-logloss:0.18254
[345]	eval-logloss:0.59219	train-logloss:0.18225
[346]	eval-logloss:0.59154	train-logloss:0.18200
[347]	eval-logloss:0.59531	train-logloss:0.18180
[348]	eval-logloss:0.59545	train-logloss:0.18162
[349]	eval-logloss:0.59542	train-logloss:0.18147
[350]	eval-logloss:0.59869	train-logloss:0.18118
[351]	eval-logloss:0.59869	train-logloss:0.18098
[352]	eval-logloss:0

2551


In [5]:
if(PREDICCION_REAL): sys.exit()

SystemExit: 

C:\Users\gianb\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
entrenamiento_label = (entrenamiento['Stage'] == 'Closed Won').astype(int)
validacion_label    = (validacion['Stage'] == 'Closed Won').astype(int)
test_label          = (test['Stage'] == 'Closed Won').astype(int)

set_entrenamiento = xgb.DMatrix(entrenamiento.drop(columns=['Stage','Fecha']),label = entrenamiento_label)
set_validacion = xgb.DMatrix(validacion.drop(columns=['Stage','Fecha']),label = validacion_label)
set_test  = xgb.DMatrix(test .drop(columns=['Stage','Fecha']),label = test_label)
evaluacion = [(set_validacion, 'eval'), (set_entrenamiento, 'train')]

modelo = xgb.train(PARAMETROS, set_entrenamiento, RONDAS, evaluacion, early_stopping_rounds=EARLY)

In [ ]:
prediccion = modelo.predict(set_test)

resultados = test[['Opportunity_ID']].copy()
resultados['Target'] = pd.Series(prediccion)
#resultados = resultados.groupby('Opportunity_ID').mean()
#resultados = resultados.reset_index()
#resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))

resultados.to_csv("prediccion.csv", index=False)
print(resultados['Target'].count())

prediccion = [1 if i > .5 else 0 for i in prediccion]

def metricas(objetivo, prediccion):
    matriz_conf = confusion_matrix(objetivo, prediccion)
    score = accuracy_score(objetivo, prediccion)
    reporte = classification_report(objetivo, prediccion)
    metricas = [matriz_conf, score, reporte]
    return(metricas)

metrics = metricas(test_label, prediccion)
print("Prediccion Test")
[print(i) for i in metrics]
print(skl.metrics.log_loss(test_label,prediccion))

prediccion = modelo.predict(set_validacion)
prediccion = [1 if i > .5 else 0 for i in prediccion]
metrics = metricas(validacion_label, prediccion)
print("Prediccion Validacion")
[print(i) for i in metrics]
print(skl.metrics.log_loss(validacion_label,prediccion))

prediccion = modelo.predict(set_entrenamiento)
prediccion = [1 if i > .5 else 0 for i in prediccion]
metrics = metricas(entrenamiento_label, prediccion)
print("Prediccion Train")
[print(i) for i in metrics]
print(skl.metrics.log_loss(entrenamiento_label,prediccion))

In [ ]:
if(not MAXIMIZAR_HIPERPARAMETROS): sys.exit()

In [ ]:
min_log_loss = 1000
min_depth = 1000
min_eta = 1000
min_rondas = 1000
rondas = 500
min_early = 1000
min_gamma = 1000
for depth in range(2,5):
    for eta in [0.1,0.3,0.5,0.7,0.9]:
        for early in range(1,5):
            for gamma in range(0,300,100):
                parametros = {"booster":"gbtree", "max_depth":depth, "eta": eta, "objective": "binary:logistic", "nthread":2,"gamma":gamma}
                modelo = xgb.train(parametros, set_entrenamiento, rondas, evaluacion,early_stopping_rounds = early)
                prediccion = modelo.predict(set_validacion)
                prediccion = [1 if i > .5 else 0 for i in prediccion]
                log_loss = skl.metrics.log_loss(validacion_label,prediccion)
                if (log_loss < min_log_loss):
                    min_log_loss = log_loss
                    min_depth = depth
                    min_eta = eta
                    min_rondas = rondas
                    min_gamma = gamma
                    min_early = early


print("log: ",min_log_loss)                
print("depth: ",min_depth)      
print("eta: ",min_eta)    
print("rondas",min_rondas)
print("gamma: ", min_gamma)
print("early: ",min_early)

In [ ]:
parametros = {"booster":"gbtree", "max_depth":min_depth, "eta": min_eta, "objective": "binary:logistic", "nthread":2,"gamma":min_gamma}
modelo = xgb.train(parametros, set_entrenamiento, min_rondas, evaluacion,early_stopping_rounds = min_early)
prediccion = modelo.predict(set_test)
prediccion = [1 if i > .5 else 0 for i in prediccion]
metrics = metricas(test_label, prediccion)
print("Prediccion Test")
[print(i) for i in metrics]
print(skl.metrics.log_loss(test_label,prediccion))